# Лабораторная работа №5: Тематическое моделирование

---
---

# Задание:
1. Дана коллекция текстовых документов `2021_SPORT`
2. Необходимо провести тематическое моделирование коллекции методами латентного размещения (аллокации) Дирихле и неотрицательного матричного разложения и визуализировать темы с разным количеством топ-слов, реализовав функцию `plot_top_words`.
    1. Для тематического моделирования понадобится `matplotlib`, `sklearn.feature_extraction.text` (`TfidfVectorizer`, `CountVectorizer`), `sklearn.decomposition` (`NMF`, `LatentDirichletAllocation`)
    2. Для препроцессинга нужно воспользоваться материалами прошлой лабораторной (в части стоп-слов, токенизации и лемматизации)
    3. Затем построить через `CountVectorizer`, `TfidfVectorizer` (попробуйте оба) векторное представление каждого текста (для визуализации полезно также сделать что-то вроде `tfidf_feature_names = tfidf_vectorizer.get_feature_names()`)
    4. Полученные вектора передавать уже в модель для тематического моделирования
    5. Количество признаков — начать с 1000, топиков — 10, топ-слов — 20

---
---

In [55]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import os
import nltk
import pymorphy3

from dataclasses import dataclass, field    
from nltk.tokenize import word_tokenize                           
from typing import List

# Создадим структуру для хранения данных
@dataclass   
class Folders: 
    files_name: List[str] = field(default_factory=list)
    data: List[str] = field(default_factory=list)

@dataclass 
class Home:
    path: str = ''
    name: str = ''
    folders_name: List[Folders] = field(default_factory=list)

# Создадим функцию для токенизации текста
def preprocess_text(text):
    nltk.download('punkt')
    nltk.download('stopwords')
    stop_words = set(nltk.corpus.stopwords.words('russian'))
    morph = pymorphy3.MorphAnalyzer()    
    tokens = word_tokenize(text)
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token.isalnum() and not token.isdigit()]
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Функция для получения корпуса слов
def get_corpus():
    home = Home()
    folders = Folders()
    corpus = []
    path_to_file_list = []
    home.path = '/home/kirill/projects/Methods-and-algorithms-for-weakly-structured-data/Methods-and-algorithms-for-weakly-structured-data/lab4/'
    home.name = '2021_SPORT'
    folders_list = os.listdir(home.path + '/' + home.name)

    for i in folders_list:
        home.folders_name.append(i)
        d = os.listdir(home.path + '/' + home.name + '/' + i) 
        folders.files_name.append(d)
        
        for el in folders.files_name:
            for j in el:
                path_to_file_list.append(home.path + '/' + home.name + '/' + i  + '/' + j)
    
    for i in path_to_file_list:
        with open(i, 'r', encoding='utf-8') as f:
            text = f.read()
            text = preprocess_text(text)
            corpus.append(text)
            
    return corpus

# Функция для отображения графиков
def plot_top_words(model, feature_names, n_top_words, titul):
    for topic_idx, topic in enumerate(model.components_):
        message = "Топик #%d: " % topic_idx
        top_words_idx = topic.argsort()[:-n_top_words - 1:-1]
        message += " ".join([feature_names[i] for i in top_words_idx])
        print(message)
        sorted_topic = topic[top_words_idx]
        plt.figure(figsize=(12, 6))
        plt.bar(range(len(top_words_idx)), sorted_topic)
        plt.ylabel('Частота слов')
        plt.xlabel('Слова')
        plt.title(titul + 'Топик {}'.format(topic_idx))
        plt.xticks(range(len(top_words_idx)), [feature_names[i] for i in top_words_idx], rotation=90)
        plt.show()
    print()
    
def Change_vector_representation_method(method):
    if method == 'tfidf_vectorizer':
        vectorizer = TfidfVectorizer(max_features=n_features)
    elif method == 'count_vectorizer':
        vectorizer = CountVectorizer(max_features=n_features)
    return vectorizer
    

n_features = 1000
n_components = 10
n_top_words = 20

corpus = get_corpus()

# Построение векторного представления
vectorizer = Change_vector_representation_method('count_vectorizer')
vectors = vectorizer.fit_transform(corpus)

nmf = NMF(
    n_components=n_components, 
    random_state=1,
    ).fit(vectors)

vectors_feature_names = vectorizer.get_feature_names_out()
plot_top_words(nmf, vectors_feature_names, n_top_words, '\nТопики NMF модели\n')

lda = LatentDirichletAllocation(
    n_components=n_components, 
    max_iter=5, learning_method='online', 
    learning_offset=50., 
    random_state=0
    ).fit(vectors)

vectors_feature_names = vectorizer.get_feature_names_out()
plot_top_words(lda, vectors_feature_names, n_top_words, '\nТопики LDA модели\n')
        


[nltk_data] Downloading package punkt to /home/kirill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kirill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kirill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kirill/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kirill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading 

TypeError: __init__() got an unexpected keyword argument 'alpha'